In [1]:
import intel_extension_for_pytorch as ipex

/opt/intel/oneapi/intelpython/envs/pytorch-gpu/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
import torch
import torch.nn as nn
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

2024-04-02 20:39:50.111971: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 20:39:50.238312: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 20:39:50.790759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 20:39:50.790818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 20:39:50.917702: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [4]:
clean = pd.read_csv('Cleaned_Indian_Food_Dataset.csv').head(1000)

In [5]:
model_name = 'gpt2'

In [6]:
model_save_path = './100GPT1api'

In [7]:
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50260, 768)

In [8]:
train_size = 0.7
max_length = 512

In [9]:
tokenizer.save_pretrained(model_save_path)

('./100GPT1api/tokenizer_config.json',
 './100GPT1api/special_tokens_map.json',
 './100GPT1api/vocab.json',
 './100GPT1api/merges.txt',
 './100GPT1api/added_tokens.json',
 './100GPT1api/tokenizer.json')

In [10]:
tokenizer.convert_tokens_to_ids(['<|pad|>'])

[50259]

In [11]:
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(**inputs, max_length=max_length, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    print(tokenizer.decode(output[0]))

In [12]:
clean = clean.sample(frac=1)
clean.reset_index(drop=True, inplace=True)


In [13]:
train_len = int(train_size * len(clean))
clean = clean[:train_len]

In [14]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|unknown|>',
 'pad_token': '<|pad|>'}

In [15]:
tokenizer.convert_tokens_to_ids(['<|startoftext|>'],)

[50257]

In [18]:

clean = clean.sample(frac=1)
clean.reset_index(drop=True,inplace=True)

In [19]:
def print_recipe(idx):
    print(f"{clean['ingredients'][idx]}\n\n{clean['instructions'][idx]}")

In [20]:
def form_string(ingredient, instruction):
    s = f"Ingredients:\n{ingredient.strip()}\n\nInstructions:\n{instruction.strip()}"
    return s

In [21]:
print(clean.columns)

Index(['TranslatedRecipeName', 'TranslatedIngredients', 'TotalTimeInMins',
       'Cuisine', 'TranslatedInstructions', 'URL', 'Cleaned-Ingredients',
       'image-url', 'Ingredient-count'],
      dtype='object')


In [22]:
data = clean.apply(lambda x: form_string(x['Cleaned-Ingredients'], x['TranslatedInstructions']), axis=1).to_list()

In [23]:
train_size = 0.85
train_len = int(train_size * len(data))
train_data = data[:train_len]
val_data = data[train_len:]

In [24]:
class RecipeDataset:
    def __init__(self, data):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for item in tqdm(data):
            encodings = tokenizer.encode_plus(item,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=max_length,  # Adjusted max_length
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'], 0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'], 0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [25]:
train_ds = RecipeDataset(data)

  0%|          | 0/700 [00:00<?, ?it/s]

In [26]:
batch_size = 2

In [27]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[0] for item in batch])
    }


In [28]:
train_ds = RecipeDataset(train_data)
val_ds = RecipeDataset(val_data)

  0%|          | 0/595 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

In [33]:
args = TrainingArguments(output_dir=model_save_path,
                         per_device_train_batch_size=batch_size,
                         per_device_eval_batch_size=batch_size,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=10,
                         save_strategy='no',
                         use_ipex=True
                        )

In [34]:
optim = torch.optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim, 20, eta_min=1e-7)

In [35]:
trainer = Trainer(model,
                  args,
                  train_dataset=train_ds,
                  eval_dataset=train_ds,  # Using train_ds for evaluation for now, change it as needed
                  data_collator=collate_fn,
                  optimizers=(optim, scheduler)
                 )

In [36]:
trainer.train()

Step,Training Loss
500,1.489900
1000,1.296900


TrainOutput(global_step=1490, training_loss=1.3240393721817323, metrics={'train_runtime': 233.2246, 'train_samples_per_second': 25.512, 'train_steps_per_second': 6.389, 'total_flos': 1554687590400000.0, 'train_loss': 1.3240393721817323, 'epoch': 10.0})

In [37]:
trainer.save_model()

In [42]:
model_save_path = "gptmodel/"
trainer.save_model(model_save_path )